<a href="https://colab.research.google.com/github/prateeshreddy/Text-Summarization-Using-Advanced-NLP-Methods/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pathlib
from zipfile import ZipFile
import xml.etree.ElementTree as ET
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!nvidia-smi

Wed Jun 16 23:09:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
question=str(input('Please Enter the question?'))

Please Enter the question?what are common adverse reactions 


# Data Preprocessing

In [ ]:
extension = ".xml";raw=[];
for item in os.listdir(r'/content/drive/MyDrive/MS_DS_Files/LAIDEL/Adverse_Reactions/Adverse_Reactions_unzipped/'): 
    if item.endswith(extension):
        data=''
        tree = ET.parse(r'/content/drive/MyDrive/MS_DS_Files/LAIDEL/Adverse_Reactions/Adverse_Reactions_unzipped/'+item)
        root=tree.getroot()
        element1=root.find('{urn:hl7-org:v3}component')
        for component in element1[0]:
            section=component[0]
            for element4 in section:
                if element4.tag == '{urn:hl7-org:v3}title':
                    if element4.text is not None:
                        if 'ADVERSE REACTIONS' in element4.text:
                            for k in section:
                                data=data+' '+(str(ET.tostring(k, encoding='utf-8', method='text').decode("utf-8")).strip())
        raw.append(data)

In [ ]:
kl=raw[0].split('\n'); huhu=[]
for rot in raw:
    non_empty_lines=[];haha='';
    for klw in rot.split('\n'):
        lines = klw.split("\n")
        non_empty_lines = [line for line in lines if line.strip() != ""]
        if len(non_empty_lines)!=0:
            if len(non_empty_lines[0].strip())>2:
                haha=haha+' '+non_empty_lines[0].strip()
    huhu.append(haha)

In [ ]:
corpusbig=[];
for k1 in huhu:
    corpus=''
    for sen in k1.split('.'):
        if ('adverse' in sen.lower()) or ('common' in sen.lower()) or ('most' in sen.lower()):
            corpus=corpus+'. '+sen
    corpusbig.append(corpus)
        

In [ ]:
import numpy as np
import pandas as pd
import nltk
#nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
target=''
for element in corpusbig:
    target=target+'.'+element    

In [ ]:
target

'..  6 ADVERSE REACTIONS The following serious adverse reactions to COUMADIN are discussed in greater detail in other sections of the labeling: Hemorrhage [see Boxed Warning Warnings and Precautions (5. 3) Other adverse reactions to COUMADIN include: Immune system disorders: hypersensitivity/allergic reactions (including urticaria and anaphylactic reactions) Vascular disorders: vasculitis Hepatobiliary disorders: hepatitis, elevated liver enzymes.  Gastrointestinal disorders: nausea, vomiting, diarrhea, taste perversion, abdominal pain, flatulence, bloating Skin disorders: rash, dermatitis (including bullous eruptions), pruritus, alopecia Respiratory disorders: tracheal or tracheobronchial calcification General disorders: chills  Most common adverse reactions to COUMADIN are fatal and nonfatal hemorrhage from any tissue or organ.  (6) To report SUSPECTED ADVERSE REACTIONS, contact Bristol-Myers Squibb at 1-800-721-5072 or FDA at 1-800-FDA-1088 or www..  6 ADVERSE REACTIONS The followin

In [ ]:
from nltk.tokenize import sent_tokenize
sentences = []
sentences.append(sent_tokenize(target))

In [ ]:
sentences = [y for x in sentences for y in x]

In [ ]:
npsize = np.array(sentences)
npsize.shape

(437,)

In [ ]:
sentences[:3]

['..  6 ADVERSE REACTIONS The following serious adverse reactions to COUMADIN are discussed in greater detail in other sections of the labeling: Hemorrhage [see Boxed Warning Warnings and Precautions (5.',
 '3) Other adverse reactions to COUMADIN include: Immune system disorders: hypersensitivity/allergic reactions (including urticaria and anaphylactic reactions) Vascular disorders: vasculitis Hepatobiliary disorders: hepatitis, elevated liver enzymes.',
 'Gastrointestinal disorders: nausea, vomiting, diarrhea, taste perversion, abdominal pain, flatulence, bloating Skin disorders: rash, dermatitis (including bullous eruptions), pruritus, alopecia Respiratory disorders: tracheal or tracheobronchial calcification General disorders: chills  Most common adverse reactions to COUMADIN are fatal and nonfatal hemorrhage from any tissue or organ.']

# Text Rank Algorithm 

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2021-06-16 23:34:48--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-06-16 23:34:48--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-06-16 23:34:48--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
# Extract word vectors
word_embeddings = {}
f = open(r'glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
len(word_embeddings)

400000

In [ ]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [ ]:
sim_mat = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [ ]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
for i in range(3):
    print(ranked_sentences[i][1])

Table 37: Adverse Reactions Occurring in ≥20% of Patients Receiving KEYTRUDA with Chemotherapy in KEYNOTE-355 Adverse Reaction KEYTRUDA200 mg every 3 weekswith chemotherapyn=596 Placebo every 3 weekswith chemotherapyn=281 All GradesGraded per NCI CTCAE v4..  6 ADVERSE REACTIONS The following clinically significant adverse reactions are described in greater detail in the Warnings and Precautions section of labeling: Mortality Imbalance in Patients with Community-Acquired Bacterial Pneumonia [see Warnings and Precautions (5.
Less Common Adverse Reactions The following adverse reactions occurred in <2% of all voriconazole-treated patients in all therapeutic studies (N=1655).
Adverse Reactions Percentage of Patients Reporting Event 6 Week Trial% Schizophrenia Patients 3 Week Trial% Bipolar Patients Olanzapine(N=72) Placebo(N=35) Olanzapine(N=107) Placebo(N=54) Sedationa Weight increased Headache Increased appetite Dizziness Abdominal painb Pain in extremity Fatigue Dry mouth Adverse Reacti

**Text Rank Algorithm Output :**
Adverse Reactions Occurring in ≥20% of Patients Receiving KEYTRUDA with Chemotherapy in KEYNOTE-355 Adverse Reaction KEYTRUDA200 mg every 3 weeks with chemotherapyn=596 Placebo every 3 weeks with chemotherapyn=281 All Grades Graded per NCI CTCAE v4.. ADVERSE REACTIONS The following clinically significant adverse reactions are described in greater detail in the Warnings and Precautions section of labeling: Mortality Imbalance in Patients with Community-Acquired Bacterial Pneumonia [see Warnings and Precautions (5.
Less Common Adverse Reactions The following adverse reactions occurred in <2% of all voriconazole-treated patients in all therapeutic studies (N=1655).
Adverse Reactions Percentage of Patients Reporting Event 6 Week Trial% Schizophrenia Patients 3 Week Trial% Bipolar Patients Olanzapine(N=72) Placebo(N=35) Olanzapine(N=107) Placebo(N=54) Sedationa Weight increased Headache Increased appetite Dizziness Abdominal painb Pain in extremity Fatigue Dry mouth Adverse Reactions Occurring at an Incidence of 2% or More among Oral Olanzapine-Treated Patients in Short-Term (3-6 weeks), Placebo-Controlled Trials Adverse reactions in adolescent patients treated with oral olanzapine (doses ≥2).



# T5 Algorithm

In [ ]:
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

In [ ]:
tokenized_text_list=[]
for i in range(len(data)):
    tokenized_text = tokenizer.encode("summarize"+data[i], return_tensors="pt").to(device)
    if len(tokenized_text)>512:
        tokenized_text_list.append(tokenized_text[:512])
    else:
        tokenized_text_list.append(tokenized_text)
tokenized_text_list

In [ ]:
punc = ",.!@#%$^&*()[]{}<>?/\|`~''"""
for i in range(len(data)):
    data[i]=re.sub("\s\s+", " ", data[i])
data

[" 6 ADVERSE REACTIONS The following clinically significant adverse reactions are described in greater detail in the Warnings and Precautions section of labeling: Mortality Imbalance in Patients with Community-Acquired Bacterial Pneumonia [see Warnings and Precautions (5.1)] Tooth Development and Enamel Hypoplasia [see Warnings and Precautions (5.2)] Inhibition of Bone Growth [see Warnings and Precautions (5.3)] Hypersensitivity Reactions [see Warnings and Precautions (5.4)] Tetracycline Class Effects [see Warnings and Precautions (5.6] The most common adverse reactions (incidence ≥2%) are nausea, vomiting, infusion site reactions, alanine aminotransferase increased, aspartate aminotransferase increased, gamma-glutamyl transferase increased, hypertension, headache, diarrhea, insomnia, and constipation. (6.1) To report SUSPECTED ADVERSE REACTIONS, contact Paratek Pharmaceuticals, Inc. at 1-833-727-2835 or FDA at 1-800-FDA-1088 or www.fda.gov/medwatch. 6.1 Clinical Trials Experience Beca

In [ ]:
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')
print(tokenizer)

PreTrainedTokenizer(name_or_path='t5-small', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>'

In [ ]:
tokenized_text_list=[]
for i in range(len(data)):
    tokenized_text = tokenizer.encode("summarize"+data[i], return_tensors="pt").to(device)
    if len(tokenized_text)>512:
        tokenized_text_list.append(tokenized_text[:512])
    else:
        tokenized_text_list.append(tokenized_text)
tokenized_text_list

Token indices sequence length is longer than the specified maximum sequence length for this model (2183 > 512). Running this sequence through the model will result in indexing errors


[tensor([[21603,   431,  8502,  ...,  1225,  6286,     1]]),
 tensor([[21603,   431,  8502,  ...,    61,  4275,     1]]),
 tensor([[21603,   431,  8502, 26794,   427,  4083, 30518,   134,    37,   826,
           2261, 11233, 14081,    12,  2847,  6122,  6762,  3162,    33,  5172,
             16,  2123,  2736,    16,   119,  6795,    13,     8,  3783,    53,
             10,   216,  2528,    52,   107,   545,   784,  2338,  5179,    15,
             26, 26826,     3,     6, 26826,     7,    11,  1266,   658, 14548,
              7,    41, 20519,    61,     3,     6,    11,  2035,    26,    32,
              7,   545,    41, 16968,     3,   908,  1484,  2771,     7,   159,
             13,  1133,    11,   119, 17451,   784,  2338, 26826,     7,    11,
           1266,   658, 14548,     7,    41, 24342,    61,     3,   908,  2149,
            447,    44,   760,    32,    15,    51,  4243,    23,    11, 16462,
           2179,    15,    51,  4243,    23,   784,  2338, 26826,     7,    11

In [ ]:
summary_ids = model.generate(tokenized_text_list[0],num_beams=4,min_length=30,length_penalty=3.0,max_length=512)
output=tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print ("\n\nSummarized text: \n",output)



Summarized text: 
 most common adverse reactions occurring in 2% of patients receiving NUZYRA in Trial 1. most common adverse reactions occurring in 2% of patients receiving NUZYRA in Trial 1. most common adverse reactions occurring in 2% of patients receiving NUZYRA in Trial 1.


In [ ]:
summary_ids = model.generate(tokenized_text_list[3],num_beams=4,min_length=30,length_penalty=3.0,max_length=512)
output=tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print ("\n\nSummarized text: \n",output)



Summarized text: 
 ciprofloxacin-treated patients were more likely to report more than one adverse reaction and on more than one occasion compared to the control group. ciprofloxacin-treated patients were more likely to report more than one adverse reaction and on more than one occasion compared to the control group. fluoroquinolones, including CIPRO, have been associated with disabling and potentially irreversible serious adverse reactions that have occurred together (5.1), including: nausea, diarrhea, liver function tests abnormal


# Bart Algorithm

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig

In [ ]:
tokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [ ]:
# Encoding the inputs and passing them to model.generate()
for data in corpusbig:
    inputs = tokenizer.batch_encode_plus([data],return_tensors='pt',max_length=1000,truncation=True)

    summary_ids = model.generate(inputs['input_ids'], early_stopping=True)
    bart_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(bart_summary)

Most common adverse reactions are: Rheumatoid and Psoriatic Arthritis, upper respiratory tract infection, nasopharyngitis, diarrhea, and headache. The proportion of patients who discontinued treatment due to any adverse reaction during the 0 to 3 months exposure was 4% for patients taking XELJANZ and 3% for placebo-treated patients. The most common types of malignancy were lung and breast cancer.
Most common adverse reaction (3% greater than placebo) to amlodipine is edema. Most common adverse reactions leading to atorvastatin discontinuation were myalgia and diarrhea. Edema, flushing, palpitations, and somnolence appear to be more common in women than in men. 5% of the patients on placebo discontinued because of adverse reactions regardless of causality.
Most common adverse reactions were fatigue, musculoskeletal pain, decreased appetite, pruritus, diarrhea, nausea, rash, pyrexia, cough, dyspnea, constipation, pain, and abdominal pain. Adverse reactions leading to permanent discontinu

# GPT-2 Algorithm

In [ ]:
from transformers import GPT2Tokenizer,GPT2LMHeadModel
tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
model=GPT2LMHeadModel.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
for lol in corpusbig:
    inputs=tokenizer.batch_encode_plus([lol],return_tensors='pt',max_length=1000)
    summary_ids=model.generate(inputs['input_ids'],early_stopping=True)
    GPT_summary=tokenizer.decode(summary_ids[0],skip_special_tokens=True)
    print(GPT_summary)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1000, but ``max_length`` is set to 20.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1000, but ``max_length`` is set to 20.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


.  6 ADVERSE REACTIONS The following clinically significant adverse reactions are described elsewhere in the labeling: Serious Infections [see Warnings and Precautions (5. 7)]  Most common adverse reactions are: Rheumatoid and Psoriatic Arthritis: Reported during the first 3 months in rheumatoid arthritis controlled clinical trials and occurring in ≥2% of patients treated with XELJANZ monotherapy or in combination with DMARDs: upper respiratory tract infection, nasopharyngitis, diarrhea, and headache. 1) To report SUSPECTED ADVERSE REACTIONS, contact Pfizer, Inc. 1 Clinical Trials Experience Because clinical studies are conducted under widely varying conditions, adverse reaction rates observed in the clinical studies of a drug cannot be directly compared to rates in the clinical studies of another drug and may not predict the rates observed in a broader patient population in clinical practice.  All seven protocols included provisions for patients taking placebo to receive treatment wit

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


.  6 ADVERSE REACTIONS The following serious adverse reactions are discussed in greater detail in other sections of the label: Myopathy and Rhabdomyolysis [see Warnings and Precautions (5. 3)]  Most common adverse reaction (3% greater than placebo) to amlodipine is edema (6.  Most common adverse reactions leading to atorvastatin discontinuation were myalgia and diarrhea (6.  To report SUSPECTED ADVERSE REACTIONS, contact Pfizer Inc. 1	Clinical Trials Experience Because clinical trials are conducted under widely varying conditions, adverse reaction rates observed in the clinical trials of a drug cannot be directly compared to rates in the clinical trials of another drug and may not reflect the rates observed in practice.  For the most part, adverse reactions have been mild or moderate in severity.  In clinical trials with CADUET, no adverse reactions peculiar to this combination have been observed.  Adverse reactions are similar in terms of nature, severity, and frequency to those repor

Input length of input_ids is 1000, but ``max_length`` is set to 20.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


# Our New Approach

## => BART QnA + BART

In [ ]:
tokenizer = BartTokenizer.from_pretrained('a-ware/bart-squadv2')
model = BartForQuestionAnswering.from_pretrained('a-ware/bart-squadv2')

In [ ]:
question=str(input('Please Enter the question?'))

Please Enter the question?what are common adverse reactions 


In [ ]:
for corpus in corpusbig:
    text_array=[]
    text=''
    for sentence in corpus.split('.'):
    #print(sentence)
        if len(text)<1950:
            text+=sentence+'.'
        else:
            text_array.append(text)
            text=''
    answer_array=[]
    for text in text_array:
        question, text = question, text
        encoding = tokenizer(question, text, return_tensors='pt')
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']

        outputs = model(input_ids, attention_mask=attention_mask, output_attentions=False)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

        all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
        answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
        answer = tokenizer.convert_tokens_to_ids(answer.split())

        answer = tokenizer.decode(answer)
        if answer != '':
            answer_array.append(answer)
    summary=''
    for sentence in corpus.split('.'):
        for answer in answer_array:
            if answer in sentence and sentence not in summary:
                summary+=sentence+'.'
    tokenizer2=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model2=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    inputs2 = tokenizer2.batch_encode_plus([xml],return_tensors='pt',max_length=1000,truncation=True)

    summary_ids2 = model2.generate(inputs2['input_ids'], early_stopping=True)
    bart_summary = tokenizer2.decode(summary_ids2[0], skip_special_tokens=True)
    print(bart_summary)
    

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

The following serious adverse reactions are described elsewhere in the labeling. To report SUSPECTED ADVERSE REACTIONS, contact Takeda Pharmaceuticals at 1-800-828-2088 or FDA at 1/800-FDA-1088 or www.fda.gov/medwatch.
The following serious adverse reactions are described elsewhere in the labeling. To report SUSPECTED ADVERSE REACTIONS, contact Takeda Pharmaceuticals at 1-800-828-2088 or FDA at 1/800-FDA-1088 or www.fda.gov/medwatch.
The following serious adverse reactions are described elsewhere in the labeling. To report SUSPECTED ADVERSE REACTIONS, contact Takeda Pharmaceuticals at 1-800-828-2088 or FDA at 1/800-FDA-1088 or www.fda.gov/medwatch.
The following serious adverse reactions are described elsewhere in the labeling. To report SUSPECTED ADVERSE REACTIONS, contact Takeda Pharmaceuticals at 1-800-828-2088 or FDA at 1/800-FDA-1088 or www.fda.gov/medwatch.
The following serious adverse reactions are described elsewhere in the labeling. To report SUSPECTED ADVERSE REACTIONS, cont

In [ ]:
answer_array

['): hypotension, somnolence, fatigue, nausea, and letharg',
 '<s>',
 'What',
 '<s>',
 '<s>What',
 '<s>']

In [ ]:
for corpus in corpusbig:
    text_array=[]
    text=''
    for sentence in corpus.split('.'):
    #print(sentence)
        if len(text)<1950:
            text+=sentence+'.'
        else:
            text_array.append(text)
            text=''
    answer_array=[]
    for text in text_array:
        question, text = question, text
        encoding = tokenizer(question, text, return_tensors='pt')
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']

        outputs = model(input_ids, attention_mask=attention_mask, output_attentions=False)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

        all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
        answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
        answer = tokenizer.convert_tokens_to_ids(answer.split())

        answer = tokenizer.decode(answer)
        if answer != '':
            answer_array.append(answer)
    summary=''
    for sentence in corpus.split('.'):
        for answer in answer_array:
            if answer in sentence and sentence not in summary:
                summary+=sentence+'.'
    tokenizer2=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model2=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    inputs2 = tokenizer2.batch_encode_plus([xml],return_tensors='pt',max_length=1000,truncation=True)

    summary_ids2 = model2.generate(inputs2['input_ids'], early_stopping=True)
    bart_summary = tokenizer2.decode(summary_ids2[0], skip_special_tokens=True)
    print(bart_summary)
    

The following serious adverse reactions are described elsewhere in the labeling. To report SUSPECTED ADVERSE REACTIONS, contact Takeda Pharmaceuticals at 1-800-828-2088 or FDA at 1/800-FDA-1088 or www.fda.gov/medwatch.
The following serious adverse reactions are described elsewhere in the labeling. To report SUSPECTED ADVERSE REACTIONS, contact Takeda Pharmaceuticals at 1-800-828-2088 or FDA at 1/800-FDA-1088 or www.fda.gov/medwatch.
The following serious adverse reactions are described elsewhere in the labeling. To report SUSPECTED ADVERSE REACTIONS, contact Takeda Pharmaceuticals at 1-800-828-2088 or FDA at 1/800-FDA-1088 or www.fda.gov/medwatch.
The following serious adverse reactions are described elsewhere in the labeling. To report SUSPECTED ADVERSE REACTIONS, contact Takeda Pharmaceuticals at 1-800-828-2088 or FDA at 1/800-FDA-1088 or www.fda.gov/medwatch.
The following serious adverse reactions are described elsewhere in the labeling. To report SUSPECTED ADVERSE REACTIONS, cont

In [ ]:
answer_array

['): hypotension, somnolence, fatigue, nausea, and letharg',
 'what',
 ': nausea, diarrhea, vomiting, and depression (depressed mood, depression, depressive symptom',
 ': irritability, vomiting, asthma (asthma, bronchospasm, wheezing), and enuresis (enuresis, nocturia, urinary incontinence',
 'what are common adverse reactions </s></s>5 mmHg, -5.4 mmHg and -8.2 mmHg. For these respective fixed doses the maximum mean changes in seated diastolic blood pressure were -3.4 mmHg, -3.3 mmHg, -4.4 mmHg and -5.4 mmHg. For these respective fixed doses the maximum mean changes in seated pulse were -4.8 bpm, -3.1 bpm, -6.5 bpm and -8.6 bpm. Decreases in blood pressure and heart rate were usually modest and asymptomatic; however, hypotension and bradycardia can occur']